In [ ]:
# ✅ Install required dependencies (run once per environment)
%pip install ultralytics opencv-python pandas matplotlib tqdm


In [ ]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU detected")


In [ ]:
import os
import cv2
import shutil
import random
import zipfile
from ultralytics import YOLO
from pathlib import Path
import matplotlib.pyplot as plt


In [1]:
# %%
import shutil
from pathlib import Path

# Paths to clean
paths_to_clean = [
    "datasets/VisDrone",
    "runs/train/visdrone_parking_detector_vehicle_only",
    "runs/train/visdrone_parking_detector",
    "runs/train/visdrone_full_dataset_retrain",
    "runs/detect",
    "runs/val"
]

for p in paths_to_clean:
    path = Path(p)
    if path.exists():
        try:
            shutil.rmtree(path, ignore_errors=True)
            print(f"🧹 Force deleted: {p}")
        except Exception as e:
            print(f"⚠️ Could not delete {p}: {e}")
    else:
        print(f"✅ Already clean: {p}")

print("\n✨ Dataset and YOLO output folders have been fully reset. Ready for a fresh setup!")


🧹 Force deleted: datasets/VisDrone
✅ Already clean: runs/train/visdrone_parking_detector_vehicle_only
✅ Already clean: runs/train/visdrone_parking_detector
✅ Already clean: runs/train/visdrone_full_dataset_retrain
🧹 Force deleted: runs/detect
✅ Already clean: runs/val

✨ Dataset and YOLO output folders have been fully reset. Ready for a fresh setup!


In [2]:
# %%
import shutil
from pathlib import Path

# Specific subfolders that block re-download
paths = [
    "datasets/VisDrone/VisDrone2019-DET-train",
    "datasets/VisDrone/VisDrone2019-DET-val",
    "datasets/VisDrone/VisDrone2019-DET-test-dev",
]

for p in paths:
    path = Path(p)
    if path.exists():
        shutil.rmtree(path, ignore_errors=True)
        print(f"🧹 Deleted: {p}")
    else:
        print(f"✅ Already clean: {p}")

print("\n✨ Old partial VisDrone folders removed. Ready to download fresh.")


🧹 Deleted: datasets/VisDrone/VisDrone2019-DET-train
🧹 Deleted: datasets/VisDrone/VisDrone2019-DET-val
🧹 Deleted: datasets/VisDrone/VisDrone2019-DET-test-dev

✨ Old partial VisDrone folders removed. Ready to download fresh.


In [1]:
# %%
import os
from pathlib import Path
from ultralytics.utils.downloads import download

# === STEP 1: Recreate core folder structure ===
base_dirs = [
    "datasets",
    "datasets/VisDrone",
    "runs",
    "runs/detect",
    "runs/train",
    "runs/val",
]

for d in base_dirs:
    Path(d).mkdir(parents=True, exist_ok=True)
    print(f"📁 Created: {d}")

# === STEP 2: Download VisDrone dataset ===
dataset_dir = Path("datasets/VisDrone")
urls = [
    "https://github.com/ultralytics/assets/releases/download/v0.0.0/VisDrone2019-DET-train.zip",
    "https://github.com/ultralytics/assets/releases/download/v0.0.0/VisDrone2019-DET-val.zip",
    "https://github.com/ultralytics/assets/releases/download/v0.0.0/VisDrone2019-DET-test-dev.zip",
]

print("\n⬇️ Downloading VisDrone dataset... (this may take a while)")
download(urls, dir=dataset_dir, threads=4)
print("\n✅ VisDrone dataset downloaded successfully!")

# === STEP 3: Verify ===
expected_dirs = [
    "VisDrone2019-DET-train",
    "VisDrone2019-DET-val",
    "VisDrone2019-DET-test-dev"
]
print("\n📦 Verifying extraction...")
for d in expected_dirs:
    path = dataset_dir / d
    if path.exists():
        print(f"✅ Found: {path}")
    else:
        print(f"❌ Missing: {path} (check if extraction failed)")

print("\n✨ Dataset structure rebuilt, VisDrone ready for YOLO conversion!")


📁 Created: datasets
📁 Created: datasets/VisDrone
📁 Created: runs
📁 Created: runs/detect
📁 Created: runs/train
📁 Created: runs/val

⬇️ Downloading VisDrone dataset... (this may take a while)
Unzipping datasets\VisDrone\VisDrone2019-DET-val.zip to C:\Users\Sean\Desktop\FINAL YEAR PROJECT\Juyptre\datasets\VisDrone\VisDrone2019-DET-val...: 100% ━━━━━━━━━━━━ 1099/1099 1.7Kfiles/s 0.7s0.0s
Unzipping datasets\VisDrone\VisDrone2019-DET-test-dev.zip to C:\Users\Sean\Desktop\FINAL YEAR PROJECT\Juyptre\datasets\VisDrone\VisDrone2019-DET-test-dev...: 100% ━━━━━━━━━━━━ 3223/3223 1.3Kfiles/s 2.5s0.1ss
Unzipping datasets\VisDrone\VisDrone2019-DET-train.zip to C:\Users\Sean\Desktop\FINAL YEAR PROJECT\Juyptre\datasets\VisDrone\VisDrone2019-DET-train...: 100% ━━━━━━━━━━━━ 12945/12945 1.4Kfiles/s 9.5ss<0.0s

✅ VisDrone dataset downloaded successfully!

📦 Verifying extraction...
✅ Found: datasets\VisDrone\VisDrone2019-DET-train
✅ Found: datasets\VisDrone\VisDrone2019-DET-val
✅ Found: datasets\VisDrone\Vis

In [3]:
# ✅ Convert VisDrone annotations to YOLO format if not done already
import shutil
from ultralytics.utils import TQDM

def visdrone2yolo(dir, split, source_name=None):
    from PIL import Image
    source_dir = dir / (source_name or f"VisDrone2019-DET-{split}")
    images_dir = dir / "images" / split
    labels_dir = dir / "labels" / split
    labels_dir.mkdir(parents=True, exist_ok=True)

    if (source_images_dir := source_dir / "images").exists():
        images_dir.mkdir(parents=True, exist_ok=True)
        for img in source_images_dir.glob("*.jpg"):
            img.rename(images_dir / img.name)

    for f in TQDM((source_dir / "annotations").glob("*.txt"), desc=f"Converting {split}"):
        img_size = Image.open(images_dir / f.with_suffix(".jpg").name).size
        dw, dh = 1.0 / img_size[0], 1.0 / img_size[1]
        lines = []

        with open(f, encoding="utf-8") as file:
            for row in [x.split(",") for x in file.read().strip().splitlines()]:
                if row[4] != "0":
                    x, y, w, h = map(int, row[:4])
                    cls = int(row[5]) - 1
                    x_center, y_center = (x + w / 2) * dw, (y + h / 2) * dh
                    w_norm, h_norm = w * dw, h * dh
                    lines.append(f"{cls} {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}\n")

        (labels_dir / f.name).write_text("".join(lines), encoding="utf-8")

splits = {"VisDrone2019-DET-train": "train", "VisDrone2019-DET-val": "val", "VisDrone2019-DET-test-dev": "test"}
for folder, split in splits.items():
    visdrone2yolo(dataset_dir, split, folder)
    shutil.rmtree(dataset_dir / folder)

print("✅ Dataset converted to YOLO format successfully.")


Converting train: ━━━━━━━━━━━━ 6471 1.2Kit/s 5.5ss
Converting val: ━━━━━━━━━━━━ 548 54.4it/s 10.7s
Converting test: ━━━━━━━━━━━━ 1610 55.6it/s 30.2s
✅ Dataset converted to YOLO format successfully.


In [4]:
yaml_path = dataset_dir / "data.yaml"

yaml_path.write_text(f"""
path: {dataset_dir}

train: images/train
val: images/val
test: images/test

names:
  0: pedestrian
  1: people
  2: bicycle
  3: car
  4: van
  5: truck
  6: tricycle
  7: awning-tricycle
  8: bus
  9: motor
""")

print("✅ data.yaml created at:", yaml_path)


✅ data.yaml created at: datasets\VisDrone\data.yaml


In [5]:
import torch
print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))


Torch version: 2.5.1+cu121
CUDA available: True
GPU: NVIDIA GeForce RTX 3060 Laptop GPU


In [6]:
# %%
import os
import torch

# Prevent stale GPU state from crashing
torch.cuda.empty_cache()

# Ensure all CUDA asserts are reported clearly
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
print("✅ CUDA debugging enabled")


✅ CUDA debugging enabled


In [7]:
#delete maybe
classes_found = set()
for split in ["train", "val", "test"]:
    path = f"{base_label_dir}/{split}"
    for file in os.listdir(path):
        if file.endswith(".txt"):
            with open(f"{path}/{file}") as f:
                for line in f:
                    c = int(line.strip().split()[0])
                    classes_found.add(c)
print("✅ Classes found in dataset:", sorted(classes_found))


NameError: name 'base_label_dir' is not defined

In [9]:
# ✅ Train YOLOv8 on VisDrone (detects vehicles and people)
from ultralytics import YOLO

model = YOLO("yolov8n.pt")
model.train(
    data=str(yaml_path),
    epochs=10,
    imgsz=640,
    batch=8,
    name="visdrone_parking_detector"
)


New https://pypi.org/project/ultralytics/8.3.215 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.214  Python-3.12.7 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=datasets\VisDrone\data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=visdrone_parking

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000002268B976D80>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([0.        , 0.001001  , 0.002002  , 0.003003  , 0.004004  ,
       0.00500501, 0.00600601, 0.00700701, 0.00800801, 0.00900901,
       0.01001001, 0.01101101, 0.01201201, 0.01301301, 0.01401401,
       0.01501502, 0.01601602, 0.01701702, 0.01801802, 0.01901902,
       0.02002002, 0.02102102, 0.02202202, 0.02302302, 0.02402402,
       0.02502503, 0.02602603, 0.02702703, 0.02802803, 0.02902903,
       0.03003003, 0.03103103, 0.03203203, 0.03303303, 0.03403403,
       0.03503504, 0.03603604, 0.03703704, 0.03803804, 0.03903904,
       0.04004004, 0.04104104, 0.04204204, 0.04304304, 0.04404404,
       0.04504505, 0.046

In [10]:
# ✅ Evaluate model performance on validation set
model.val(data=str(yaml_path))


Ultralytics 8.3.214  Python-3.12.7 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
Model summary (fused): 72 layers, 3,007,598 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 1008.4294.5 MB/s, size: 126.7 KB)
val: Scanning C:\Users\Sean\Desktop\FINAL YEAR PROJECT\Juyptre\datasets\VisDrone\labels\val.cache... 548 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 548/548  0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 35/35 3.8it/s 9.2s0.2s
                   all        548      38759      0.351       0.27      0.252      0.145
            pedestrian        520       8844      0.347      0.298      0.267      0.112
                people        482       5125       0.41      0.185      0.205     0.0692
               bicycle        364       1287      0.174     0.0551      0.039     0.0134
                   car        515      14064      0.513      0.715      0.

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000002281FA0DDF0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([0.        , 0.001001  , 0.002002  , 0.003003  , 0.004004  ,
       0.00500501, 0.00600601, 0.00700701, 0.00800801, 0.00900901,
       0.01001001, 0.01101101, 0.01201201, 0.01301301, 0.01401401,
       0.01501502, 0.01601602, 0.01701702, 0.01801802, 0.01901902,
       0.02002002, 0.02102102, 0.02202202, 0.02302302, 0.02402402,
       0.02502503, 0.02602603, 0.02702703, 0.02802803, 0.02902903,
       0.03003003, 0.03103103, 0.03203203, 0.03303303, 0.03403403,
       0.03503504, 0.03603604, 0.03703704, 0.03803804, 0.03903904,
       0.04004004, 0.04104104, 0.04204204, 0.04304304, 0.04404404,
       0.04504505, 0.046

In [11]:
# ✅ Run inference on your parking lot video
video_path = "Assets/Video1.mp4"
output_dir = "runs/detect/parking_output"

results = model.predict(source=video_path, save=True, project="runs/detect", name="parking_output", show=False)
print("🎥 Inference complete. Output saved to:", results[0].save_dir)



WARNING 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/376) c:\Users\Sean\Desktop\FINAL YEAR PROJECT\Juyptre\Assets\Video1.mp4: 384x640 2 pedestrians, 221 cars, 7 vans, 80.1ms
video 1/1 (frame 2/376) c:\Users\Sean\Desktop\FINAL YEAR PROJECT\Juyptre\Assets\Video1.mp4: 384x640 1 pedestrian, 229 cars, 7 vans, 37.1ms
video 1/1 (frame 3/376) c:\Users\Sean\Desktop\FINAL YEAR PROJECT\Juyptre\Assets\Video1.mp4: 384x640 1 pedestrian, 226 cars, 6 vans, 40.1ms
video 1/1 (frame 4/376) c:\Users\Sean\Des

In [12]:
base = "datasets/VisDrone"
val_images = os.listdir(f"{base}/images/val")
random.shuffle(val_images)
num_test = int(0.1 * len(val_images))
test_images = val_images[:num_test]

os.makedirs(f"{base}/images/test", exist_ok=True)
os.makedirs(f"{base}/labels/test", exist_ok=True)

for img in test_images:
    name = os.path.splitext(img)[0]
    shutil.move(f"{base}/images/val/{img}", f"{base}/images/test/{img}")
    if os.path.exists(f"{base}/labels/val/{name}.txt"):
        shutil.move(f"{base}/labels/val/{name}.txt", f"{base}/labels/test/{name}.txt")

print(f"✅ Created test set with {len(test_images)} images and labels.")


NameError: name 'random' is not defined

In [13]:
for split in ["train", "val", "test"]:
    img_count = len(os.listdir(f"{dataset_dir}/images/{split}"))
    label_count = len(os.listdir(f"{dataset_dir}/labels/{split}"))
    print(f"{split.capitalize()} → Images: {img_count} | Labels: {label_count}")


Train → Images: 6471 | Labels: 6471
Val → Images: 548 | Labels: 548
Test → Images: 1610 | Labels: 1610


In [14]:
import cv2
import os

# Paths
input_path = "runs/detect/parking_output/Video1.avi"   # change this if your AVI has a different name
output_path = "runs/detect/parking_output/output_fixed.mp4"

# Check input exists
if not os.path.exists(input_path):
    raise FileNotFoundError(f"Input file not found: {input_path}")

# Open video
cap = cv2.VideoCapture(input_path)
if not cap.isOpened():
    raise IOError("❌ Could not open input video.")

# Get properties
fps = cap.get(cv2.CAP_PROP_FPS)
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print(f"Video Info: {fps:.2f} FPS, {width}x{height}")

# Create MP4 writer
fourcc = cv2.VideoWriter_fourcc(*'avc1')  # Better compatibility than mp4v
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# Convert frame by frame
frame_count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break
    out.write(frame)
    frame_count += 1

# Clean up
cap.release()
out.release()

print(f"✅ Conversion complete! {frame_count} frames saved to: {output_path}")


Video Info: 24.00 FPS, 1920x1080
✅ Conversion complete! 372 frames saved to: runs/detect/parking_output/output_fixed.mp4
